In [2]:
# Imports for making dataframe
import pandas as pd
import numpy as np

In [3]:
# Imports for crawling for HTML -> JSON
import requests
import json

In [4]:
# Import for finding texts in HTML JSON
from bs4 import BeautifulSoup

In [5]:
# Imports for seeing full text reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [11]:
# Goodreads API Developer key
key = 'mTBsGSOvfBaN0OF25U5g'
secret = 'AkIOvUnGYksPPicdwWfQVStbr61O0YzzrVu1ZUmpGA'

In [6]:
# Load Amazon data to find reviews of same books from Goodreads
amazon = pd.read_csv('amazonData.csv')
amazon

,overall,reviewTime,asin,reviewText
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur..."
1,5.0,"12 27, 2012",0001713353,This was one of my favorites when I was a smal...
2,5.0,"08 15, 2011",0001713353,"So, you think you have problems? Things could ..."
3,5.0,"11 24, 2008",0001713353,My mom bought this for my oldest son. He and ...
4,3.0,"09 18, 2006",0001713353,41 years later:\n\nThe cheese is government ch...
...,...,...,...,...
115439,5.0,"11 27, 1999",0030554179,"""All animals are equal, but some are more equa..."
115440,5.0,"09 9, 1999",0030554179,It's a story about animals who take over a far...
115441,5.0,"07 27, 1999",0030554179,We should not be content to reduce this book t...
115442,5.0,"06 5, 1999",0030554179,George Orwell is the pseudonym of Eric A. Blai...


In [8]:
# Unique set of books to crawl from Goodreads
books = set(amazon.asin)
print(len(books))

3007


In [9]:
## TEST
test = set()
for i,val in enumerate(books):
    if i == 10:
        break
    test.add(val)
books = test
books

{'0002008009',
 '0002212560',
 '0006172482',
 '0007232233',
 '0007321988',
 '0007462700',
 '0020259107',
 '002045600X',
 '0024146706',
 '0030180236'}

In [14]:
# Convert ASIN (ISBN) to Goodreads internal ID for look up
res = requests.get('https://www.goodreads.com/book/isbn_to_id', params={'key': key, 'isbn': books})

In [20]:
res.url

'https://www.goodreads.com/book/isbn_to_id?key=mTBsGSOvfBaN0OF25U5g&isbn=0007462700&isbn=002045600X&isbn=0007321988&isbn=0030180236&isbn=0024146706&isbn=0020259107&isbn=0002212560&isbn=0002008009&isbn=0007232233&isbn=0006172482'

In [8]:
# Create empty dataframe to store Goodreads reviews & ratings
column_names = ['overall', 'reviewTime', 'asin', 'reviewText']
df = pd.DataFrame(columns = column_names)
print(df)

Empty DataFrame
Columns: [overall, reviewTime, asin, reviewText]
Index: []


In [10]:
# Log in Goodreads
driver = webdriver.Chrome()
driver.get ('https://www.goodreads.com/user/sign_in?source=home')
driver.find_element_by_name('user[email]').send_keys('lamtk1@uci.edu') ## ENTER EMAIL
driver.find_element_by_name('user[password]').send_keys('T7u4L9a7m') ## ENTER PASSWORD
driver.find_element_by_name('next').click()                            

In [11]:
# Set of books not found on Goodreads:
notfound = set()

In [12]:
# Get most popular reviews for each book (10 if there are more than 10 reviews, or all reviews if less)
for book in books:
    # Try to look up the book in Goodreads. If a book isn't found it will not be added to the dataset -> DATA CLEANING LATER
    try:
        print('TRYING BOOK:', book)
        res = requests.get('https://www.goodreads.com/book/isbn.json', params={'key': key, 'isbn': book, 'text_only': True})
        data = res.json()
        # Parse HTML JSON to find link to reviews
        soup = BeautifulSoup(data['reviews_widget'], 'html.parser')
        url = soup.find('iframe')['src']
        # Open link to reviews using Selenium
        #print('   Opening iframe: ', url)
        driver.get(str(url))
        #print('      Opened iframe')
        # Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        #print('   Opening tabs')
        links = driver.find_elements_by_link_text('...more') 
        count = 0
        for link in links:
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            count += 1
        #print('      Opened tabs:', count)
        for i in range(count):
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            time = time.text.strip()
            # Only save reviews before 2013
            if int(time[-4:]) < 2013:
                print('   SAVING REVIEW')
                # Use BeautifulSoup to save rating
                rating = s.find('meta', itemprop='ratingValue')
                # Use BeautifulSoup to save text review
                review = s.find('div', itemprop='reviewBody')
                # Add review to dataframe
                df = df.append({'overall':rating.get('content'), 'reviewTime':time, 'asin':book, 
                                'reviewText':review.text.strip()}, ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    except:
        print('   NOT FOUND')
        notfound.add(book)

TRYING BOOK: 0028622618
   NOT FOUND
TRYING BOOK: 0029118239
   NOT FOUND
TRYING BOOK: 0007151136
   NOT FOUND
TRYING BOOK: 0006755224
   NOT FOUND
TRYING BOOK: 0027181006
   NOT FOUND
TRYING BOOK: 0007270720
   NOT FOUND
TRYING BOOK: 0007282753
   NOT FOUND
TRYING BOOK: 0007162499
   NOT FOUND
TRYING BOOK: 0007339399
   NOT FOUND
TRYING BOOK: 0007236093
   NOT FOUND


In [13]:
df

,overall,reviewTime,asin,reviewText


In [14]:
print(notfound)

{'0028622618', '0029118239', '0007151136', '0006755224', '0007339399', '0007282753', '0007236093', '0007162499', '0027181006', '0007270720'}
